<a href="https://colab.research.google.com/github/noeyxos/Python/blob/master/PythonAzure01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install azure-storage-blob azure-identity  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.0/388.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.5/135.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.9/173.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 3.2 MB/s eta 0:00:00


In [2]:
import os, uuid
from azure.identity import DefaultAzureCredential                               #Credential [에버랜드 손목띠]
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [3]:
# Blob Services 에 접속
connect_str = ''
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

In [9]:
container_name = 'test2container'

container_client = blob_service_client.create_container(container_name)

In [5]:
# Create a local directory to hold blob data
local_path = './data'
os.mkdir(local_path)                                                            #os.mkdir(local_path)?

# Create a file in the local data directory to upload and download
local_file_name ='welcome' + '.txt'
upload_file_path = os.path.join(local_path, local_file_name)


#Write text to the file 
file = open(upload_file_path, mode='w')                                         #mode='w' 쓰기 모드로 열기 
file.write('Welcome to Python')
file.close()


In [11]:
blob_client = blob_service_client.get_blob_client(
    container = container_name,
    blob = local_file_name
)

with open(file = upload_file_path, mode='rb') as data:                          # mode='rb' bin 파일로 연다 (파일이 하나도 안깨지는 장점)      
  blob_client.upload_blob(data)

In [15]:
# blob 파일의 리스트를 확인한다. 
print('\nListing blobs')

blob_list = container_client.list_blobs()
for blob in blob_list:
  print('\t'+ blob.name)


Listing blobs
	welcome.txt


In [19]:
# Download the blob to a local file
# Add 'DOWNLOAD' before the .txt extendsion so you can see both files in the data direct

#blob storage에 업로드
download_file_path = os.path.join(local_path, 
                                  str.replace(local_file_name,'.txt','_DOWNLOAD.txt'))           #str.replace(타겟, 바꾸기 전, 바뀐 후)
print(download_file_path)

#blob_service 야 container를 가져와라 ! 
container_client = blob_service_client.get_container_client(container = container_name)

#download
with open(file=download_file_path, mode='wb') as download_file:
  download_file.write(container_client.download_blob(blob.name).readall())

./data/welcome_DOWNLOAD.txt


작업 끝나고 나서 리소스 정리할 때 사용

In [20]:
# clean Up                                                                      
print('\nPress the Enter Key to begin clean up')
input()

print('Deleting blob container...')
container_client.delete_container()

print('Deleting the local source and downloaded files...')
os.remove(upload_file_path)
os.remove(download_file_path)
os.rmdir(local_path)                                                            #rmdir removedirectory 의 약자

print('Done')



Press the Enter Key to begin clean up

Deleting blob container...
Deleting the local source and downloaded files...
Done
